# Mortgage Workflow

This example demonstrates ETL and data conversion operations on mortgage data, and is adapted from [RAPIDS Examples](https://github.com/rapidsai-community/notebooks-contrib/blob/branch-0.14/intermediate_notebooks/E2E/mortgage/mortgage_e2e.ipynb)

To acquire this dataset, please visit [RAPIDS Datasets Homepage](https://docs.rapids.ai/datasets/mortgage-data)

### Notes on running these queries:

Bodo is used by defaults, which distributes data chunks across cores automatically.

The output cells are from execution on one **m5.24xlarge** instance (24 cores, 192GiB memory) using [1 Year dataset](http://rapidsai-data.s3-website.us-east-2.amazonaws.com/notebook-mortgage-data/mortgage_2000.tgz) which is ~3.9GB. 

To run the code:
1. Download and unzip the dataset in your workspace.
2. If dataset is not stored in the same folder as the notebook, change path in loading functions to reflect your path.
2. If you want to run the example using pandas only (without Bodo):
    1. Comment lines with Jupyter parallel magic (`%%px`) and bodo decorator (`@bodo.jit`) from all the code cells.
    2. Then, re-run cells from the beginning.






In [1]:
%%px
import numpy as np
import pandas as pd
import itertools
import time
import bodo

## Load Acquistion and Performance Data

In [2]:
%%px
@bodo.jit(distributed=["acq_df"])
def load_acquisition_data(year, quarter):
    # read acquisition file
    cols = [
        "loan_id",
        "orig_channel",
        "seller_name",
        "orig_interest_rate",
        "orig_upb",
        "orig_loan_term",
        "orig_date",
        "first_pay_date",
        "orig_ltv",
        "orig_cltv",
        "num_borrowers",
        "dti",
        "borrower_credit_score",
        "first_home_buyer",
        "loan_purpose",
        "property_type",
        "num_units",
        "occupancy_status",
        "property_state",
        "zip",
        "mortgage_insurance_percent",
        "product_type",
        "coborrow_credit_score",
        "mortgage_insurance_type",
        "relocation_mortgage_indicator",
    ]
    dtypes = {
        "loan_id": np.int64,
        "orig_channel": pd.CategoricalDtype(["B", "C", "R"]),
        "seller_name": str,
        "orig_interest_rate": np.float64,
        "orig_upb": np.int64,
        "orig_loan_term": np.int64,
        "orig_date": str,
        "first_pay_date": str,
        "orig_ltv": np.float64,
        "orig_cltv": np.float64,
        "num_borrowers": np.float64,
        "dti": np.float64,
        "borrower_credit_score": np.float64,
        "first_home_buyer": pd.CategoricalDtype(["N", "U", "Y"]),
        "loan_purpose": pd.CategoricalDtype(["C", "P", "R", "U"]),
        "property_type": pd.CategoricalDtype(["CO", "CP", "MH", "PU", "SF"]),
        "num_units": np.int64,
        "occupancy_status": pd.CategoricalDtype(["I", "P", "S"]),
        "property_state": pd.CategoricalDtype(
            [
                "AK",
                "AL",
                "AR",
                "AZ",
                "CA",
                "CO",
                "CT",
                "DC",
                "DE",
                "FL",
                "GA",
                "HI",
                "IA",
                "ID",
                "IL",
                "IN",
                "KS",
                "KY",
                "LA",
                "MA",
                "MD",
                "ME",
                "MI",
                "MN",
                "MO",
                "MS",
                "MT",
                "NC",
                "ND",
                "NE",
                "NH",
                "NJ",
                "NM",
                "NV",
                "NY",
                "OH",
                "OK",
                "OR",
                "PA",
                "PR",
                "RI",
                "SC",
                "SD",
                "TN",
                "TX",
                "UT",
                "VA",
                "VI",
                "VT",
                "WA",
                "WI",
                "WV",
                "WY",
            ]
        ),
        "zip": np.int64,
        "mortgage_insurance_percent": np.float64,
        "product_type": pd.CategoricalDtype(["FRM"]),
        "coborrow_credit_score": np.float64,
        "mortgage_insurance_type": np.float64,
        "relocation_mortgage_indicator": pd.CategoricalDtype(["N", "Y"]),
    }
    acq_file = "mortgage/acq/Acquisition_" + str(year) + "Q" + str(quarter) + ".txt"
    acq_df = pd.read_csv(
        acq_file, names=cols, delimiter="|", dtype=dtypes, parse_dates=[6, 7]
    )
    return acq_df

In [3]:
%%px
@bodo.jit(distributed=["pdf"])
def load_performance_data(perf_file):
    # read performance file
    cols = [
        "loan_id",
        "monthly_reporting_period",
        "servicer",
        "interest_rate",
        "current_actual_upb",
        "loan_age",
        "remaining_months_to_legal_maturity",
        "adj_remaining_months_to_maturity",
        "maturity_date",
        "msa",
        "current_loan_delinquency_status",
        "mod_flag",
        "zero_balance_code",
        "zero_balance_effective_date",
        "last_paid_installment_date",
        "foreclosed_after",
        "disposition_date",
        "foreclosure_costs",
        "prop_preservation_and_repair_costs",
        "asset_recovery_costs",
        "misc_holding_expenses",
        "holding_taxes",
        "net_sale_proceeds",
        "credit_enhancement_proceeds",
        "repurchase_make_whole_proceeds",
        "other_foreclosure_proceeds",
        "non_interest_bearing_upb",
        "principal_forgiveness_upb",
        "repurchase_make_whole_proceeds_flag",
        "foreclosure_principal_write_off_amount",
        "servicing_activity_indicator",
    ]
    dtypes = {
        "loan_id": np.int64,
        "monthly_reporting_period": str,
        "servicer": str,
        "interest_rate": np.float64,
        "current_actual_upb": np.float64,
        "loan_age": np.float64,
        "remaining_months_to_legal_maturity": np.float64,
        "adj_remaining_months_to_maturity": np.float64,
        "maturity_date": str,
        "msa": np.float64,
        "current_loan_delinquency_status": np.int32,
        "mod_flag": pd.CategoricalDtype(["N", "Y"]),
        "zero_balance_code": pd.CategoricalDtype(
            ["01", "02", "06", "09", "03", "15", "16"]
        ),
        "zero_balance_effective_date": str,
        "last_paid_installment_date": str,
        "foreclosed_after": str,
        "disposition_date": str,
        "foreclosure_costs": np.float64,
        "prop_preservation_and_repair_costs": np.float64,
        "asset_recovery_costs": np.float64,
        "misc_holding_expenses": np.float64,
        "holding_taxes": np.float64,
        "net_sale_proceeds": np.float64,
        "credit_enhancement_proceeds": np.float64,
        "repurchase_make_whole_proceeds": np.float64,
        "other_foreclosure_proceeds": np.float64,
        "non_interest_bearing_upb": np.float64,
        "principal_forgiveness_upb": np.float64,
        "repurchase_make_whole_proceeds_flag": pd.CategoricalDtype(["N", "Y"]),
        "foreclosure_principal_write_off_amount": np.float64,
        "servicing_activity_indicator": pd.CategoricalDtype(["N", "Y"]),
    }
    pdf = pd.read_csv(
        perf_file,
        names=cols,
        delimiter="|",
        dtype=dtypes,
        parse_dates=[1, 8, 13, 14, 15, 16],
    )
    return pdf

## Data Conversion

1. Create features denoting whether a loan_id has ever been delinquent for over 30, 90 and 180 days.
2. Compute features denoting the earliest reported date when a loan_id became delinquent for more than 30, 90 and 180 days.
3. Merge the ever and delinq features table on loan_id
4. Join the performance table with the features table. (delinq and ever features)
5. For every loan_id in a 12 month window compute a feature denoting
    whether it has been delinquent for over 3 months or had an unpaid principal balance.
6. Combines the 12_mon features table with the ever_delinq features tables
7. Combines the grouped table with all features with the original Performance table
    

In [4]:
%%px
@bodo.jit(distributed=["joined_df", "tmpdf"])
def get_tmp_df(joined_df, y):
    n_months = 12
    tmpdf = joined_df[
        ["loan_id", "timestamp_year", "timestamp_month", "delinquency_12", "upb_12"]
    ]
    tmpdf["josh_months"] = tmpdf["timestamp_year"] * 12 + tmpdf["timestamp_month"]
    tmpdf["josh_mody_n"] = np.floor(
        (tmpdf["josh_months"].astype(np.float64) - 24000 - y) / 12
    )
    tmpdf_d = tmpdf.groupby(["loan_id", "josh_mody_n"], as_index=False)[
        "delinquency_12"
    ].max()
    tmpdf_m = tmpdf.groupby(["loan_id", "josh_mody_n"], as_index=False)["upb_12"].min()
    tmpdf_d["upb_12"] = tmpdf_m["upb_12"]
    tmpdf = tmpdf_d
    tmpdf["delinquency_12"] = (tmpdf["delinquency_12"] > 3).astype(np.int32)
    tmpdf["delinquency_12"] += (tmpdf["upb_12"] == 0).astype(np.int32)
    tmpdf["timestamp_year"] = (
        ((tmpdf["josh_mody_n"] * n_months) + 24000 + (y - 1)) / 12
    ).astype(np.int32)
    tmpdf["timestamp_month"] = np.full_like(tmpdf["timestamp_year"].values, y, np.int8)
    tmpdf.drop(columns=["josh_mody_n"], inplace=True)
    return tmpdf

In [5]:
%%px
# This is the value used to fill NaN dates
dt64_fill = np.dtype("datetime64[ns]").type("1970-01-01").astype("datetime64[ns]")

In [6]:
%%px
@bodo.jit(distributed=["final_gdf"], cache=True)
def morg_func(year, quarter, perf_file):
    t1 = time.time()
    # read names file
    names_file = "mortgage/names.csv"
    names_df = pd.read_csv(
        names_file,
        delimiter="|",
        names=["seller_name", "new"],
        dtype={"seller_name": str, "new": str},
    )

    acq_df = load_acquisition_data(year, quarter)
    pdf = load_performance_data(perf_file)

    print("read time", time.time() - t1)
    t1 = time.time()

    acq_df = acq_df.merge(names_df, how="left", on=["seller_name"])
    acq_df.drop(columns=["seller_name"], inplace=True)
    acq_df["seller_name"] = acq_df["new"]
    acq_df.drop(columns=["new"], inplace=True)

    # create ever features
    everdf = pdf[["loan_id", "current_loan_delinquency_status"]]
    everdf = everdf.groupby("loan_id", as_index=False).max()
    everdf["ever_30"] = (everdf["current_loan_delinquency_status"] >= 1).astype(np.int8)
    everdf["ever_90"] = (everdf["current_loan_delinquency_status"] >= 3).astype(np.int8)
    everdf["ever_180"] = (everdf["current_loan_delinquency_status"] >= 6).astype(
        np.int8
    )
    everdf.drop(columns=["current_loan_delinquency_status"], inplace=True)

    # create delinq features
    delinq_df = pdf[
        ["loan_id", "monthly_reporting_period", "current_loan_delinquency_status"]
    ]
    delinq_30 = (
        delinq_df[delinq_df["current_loan_delinquency_status"] >= 1][
            ["loan_id", "monthly_reporting_period"]
        ]
        .groupby("loan_id", as_index=False)
        .min()
    )
    delinq_30["delinquency_30"] = delinq_30["monthly_reporting_period"]
    delinq_30.drop(columns=["monthly_reporting_period"], inplace=True)
    delinq_90 = (
        delinq_df[delinq_df["current_loan_delinquency_status"] >= 3][
            ["loan_id", "monthly_reporting_period"]
        ]
        .groupby("loan_id", as_index=False)
        .min()
    )
    delinq_90["delinquency_90"] = delinq_90["monthly_reporting_period"]
    delinq_90.drop(columns=["monthly_reporting_period"], inplace=True)
    delinq_180 = (
        delinq_df[delinq_df["current_loan_delinquency_status"] >= 6][
            ["loan_id", "monthly_reporting_period"]
        ]
        .groupby("loan_id", as_index=False)
        .min()
    )
    delinq_180["delinquency_180"] = delinq_180["monthly_reporting_period"]
    delinq_180.drop(columns=["monthly_reporting_period"], inplace=True)
    delinq_merge = delinq_30.merge(delinq_90, how="left", on=["loan_id"])
    delinq_merge["delinquency_90"] = delinq_merge["delinquency_90"].fillna(dt64_fill)
    delinq_merge = delinq_merge.merge(delinq_180, how="left", on=["loan_id"])
    delinq_merge["delinquency_180"] = delinq_merge["delinquency_180"].fillna(dt64_fill)

    # join ever delinq features
    everdf = everdf.merge(delinq_merge, on=["loan_id"], how="left")
    everdf["delinquency_30"] = everdf["delinquency_30"].fillna(dt64_fill)
    everdf["delinquency_90"] = everdf["delinquency_90"].fillna(dt64_fill)
    everdf["delinquency_180"] = everdf["delinquency_180"].fillna(dt64_fill)

    # create joined df
    test = pdf[
        [
            "loan_id",
            "monthly_reporting_period",
            "current_loan_delinquency_status",
            "current_actual_upb",
        ]
    ]
    test["timestamp"] = test["monthly_reporting_period"]
    test.drop(columns=["monthly_reporting_period"], inplace=True)
    test["timestamp_month"] = test["timestamp"].dt.month
    test["timestamp_year"] = test["timestamp"].dt.year
    test["delinquency_12"] = test["current_loan_delinquency_status"]
    test.drop(columns=["current_loan_delinquency_status"], inplace=True)
    test["upb_12"] = test["current_actual_upb"]
    test.drop(columns=["current_actual_upb"], inplace=True)
    test["upb_12"] = test["upb_12"].fillna(999999999)
    test["delinquency_12"] = test["delinquency_12"].fillna(-1)

    joined_df = test.merge(everdf, how="left", on=["loan_id"])
    joined_df["ever_30"] = joined_df["ever_30"].fillna(-1)
    joined_df["ever_90"] = joined_df["ever_90"].fillna(-1)
    joined_df["ever_180"] = joined_df["ever_180"].fillna(-1)
    joined_df["delinquency_30"] = joined_df["delinquency_30"].fillna(dt64_fill)
    joined_df["delinquency_90"] = joined_df["delinquency_90"].fillna(dt64_fill)
    joined_df["delinquency_180"] = joined_df["delinquency_180"].fillna(dt64_fill)

    joined_df["timestamp_year"] = joined_df["timestamp_year"].astype(np.int32)
    joined_df["timestamp_month"] = joined_df["timestamp_month"].astype(np.int32)

    # create_12_mon_features
    tmpdf_1 = get_tmp_df(joined_df, 1)
    tmpdf_2 = get_tmp_df(joined_df, 2)
    tmpdf_3 = get_tmp_df(joined_df, 3)
    tmpdf_4 = get_tmp_df(joined_df, 4)
    tmpdf_5 = get_tmp_df(joined_df, 5)
    tmpdf_6 = get_tmp_df(joined_df, 6)
    tmpdf_7 = get_tmp_df(joined_df, 7)
    tmpdf_8 = get_tmp_df(joined_df, 8)
    tmpdf_9 = get_tmp_df(joined_df, 9)
    tmpdf_10 = get_tmp_df(joined_df, 10)
    tmpdf_11 = get_tmp_df(joined_df, 11)
    tmpdf_12 = get_tmp_df(joined_df, 12)
    testdf = pd.concat(
        [
            tmpdf_1,
            tmpdf_2,
            tmpdf_3,
            tmpdf_4,
            tmpdf_5,
            tmpdf_6,
            tmpdf_7,
            tmpdf_8,
            tmpdf_9,
            tmpdf_10,
            tmpdf_11,
            tmpdf_12,
        ]
    )

    # combine_joined_12_mon
    joined_df.drop(columns=["delinquency_12", "upb_12"], inplace=True)
    joined_df["timestamp_year"] = joined_df["timestamp_year"].astype(np.int32)
    joined_df["timestamp_month"] = joined_df["timestamp_month"].astype(np.int8)
    joined_df = joined_df.merge(
        testdf, how="left", on=["loan_id", "timestamp_year", "timestamp_month"]
    )

    # final_performance_delinquency
    merged = pdf
    merged["timestamp_month"] = merged["monthly_reporting_period"].dt.month
    merged["timestamp_month"] = merged["timestamp_month"].astype(np.int8)
    merged["timestamp_year"] = merged["monthly_reporting_period"].dt.year
    merged["timestamp_year"] = merged["timestamp_year"].astype(np.int32)
    merged = merged.merge(
        joined_df, how="left", on=["loan_id", "timestamp_year", "timestamp_month"]
    )
    perf_df = merged.drop(columns=["timestamp_year", "timestamp_month"])

    final_gdf = perf_df.merge(acq_df, how="left", on=["loan_id"])

    final_gdf["delinquency_12"] = final_gdf["delinquency_12"] > 0
    final_gdf["delinquency_12"] = (
        final_gdf["delinquency_12"].fillna(False).astype(np.int32)
    )
    t2 = time.time()
    print("exec time", t2 - t1)
    return final_gdf

In [7]:
%%px
perf_file = "mortgage/perf/Performance_2000Q1.txt"
df = morg_func(2000, 1, perf_file)

[stdout:0] 
read time 12.119809865951538
exec time 3.6855051517486572


In [8]:
%%px
df.head()

,loan_id,monthly_reporting_period,servicer,interest_rate,current_actual_upb,loan_age,remaining_months_to_legal_maturity,adj_remaining_months_to_maturity,maturity_date,msa,...,num_units,occupancy_status,property_state,zip,mortgage_insurance_percent,product_type,coborrow_credit_score,mortgage_insurance_type,relocation_mortgage_indicator,seller_name
0,100015874399,2001-10-01,NaN,8.750,239260.74,19.0,341.0,340.0,2030-03-01,45300.0,...,1,P,FL,335,30.0,FRM,NaN,1.0,N,NaN
1,100360488003,2001-01-01,NaN,8.125,113052.04,11.0,349.0,349.0,2030-02-01,12060.0,...,1,P,GA,301,30.0,FRM,NaN,1.0,N,NaN
2,100490311107,2002-01-01,NaN,8.000,107999.40,24.0,336.0,335.0,2030-01-01,31100.0,...,1,I,CA,913,NaN,FRM,778.0,NaN,N,AmTrust
3,100507009460,2001-01-01,NaN,8.000,64476.54,9.0,171.0,165.0,2015-04-01,44100.0,...,1,P,IL,627,NaN,FRM,747.0,NaN,N,Other
4,100507009460,2002-09-01,NaN,8.000,58696.99,29.0,151.0,141.0,2015-04-01,44100.0,...,1,P,IL,627,NaN,FRM,747.0,NaN,N,Other


,loan_id,monthly_reporting_period,servicer,interest_rate,current_actual_upb,loan_age,remaining_months_to_legal_maturity,adj_remaining_months_to_maturity,maturity_date,msa,...,num_units,occupancy_status,property_state,zip,mortgage_insurance_percent,product_type,coborrow_credit_score,mortgage_insurance_type,relocation_mortgage_indicator,seller_name
369531,100267077513,2000-09-01,NaN,7.875,42757.74,8.0,352.0,352.0,2030-01-01,15940.0,...,1,P,OH,447,NaN,FRM,NaN,NaN,N,NaN
369532,100268776682,2001-11-01,NaN,7.625,58928.09,23.0,337.0,337.0,2029-12-01,38300.0,...,1,P,PA,156,NaN,FRM,NaN,NaN,N,NaN
369533,100516792990,2000-05-01,NaN,7.875,NaN,4.0,356.0,356.0,2030-01-01,12580.0,...,1,P,MD,211,NaN,FRM,798.0,NaN,N,Other
369534,100516792990,2000-08-01,NaN,7.875,226879.76,7.0,353.0,353.0,2030-01-01,12580.0,...,1,P,MD,211,NaN,FRM,798.0,NaN,N,Other
369535,100676177700,2001-01-01,NaN,8.375,40384.50,11.0,349.0,348.0,2030-02-01,33100.0,...,1,P,FL,331,30.0,FRM,NaN,1.0,N,NaN


,loan_id,monthly_reporting_period,servicer,interest_rate,current_actual_upb,loan_age,remaining_months_to_legal_maturity,adj_remaining_months_to_maturity,maturity_date,msa,...,num_units,occupancy_status,property_state,zip,mortgage_insurance_percent,product_type,coborrow_credit_score,mortgage_insurance_type,relocation_mortgage_indicator,seller_name
743289,100048724218,2001-10-01,NaN,7.750,148870.34,21.0,339.0,327.0,2030-01-01,13460.0,...,1,S,OR,977,NaN,FRM,NaN,NaN,N,NaN
743290,100329180175,2000-02-01,NaN,7.875,NaN,0.0,360.0,360.0,2030-02-01,33460.0,...,1,P,MN,550,NaN,FRM,755.0,NaN,N,NaN
743291,100329180175,2002-07-01,NaN,7.875,179895.58,29.0,331.0,331.0,2030-02-01,33460.0,...,1,P,MN,550,NaN,FRM,755.0,NaN,N,NaN
743292,100340520611,2001-04-01,NaN,8.750,172506.29,15.0,345.0,337.0,2030-01-01,49340.0,...,1,P,MA,16,30.0,FRM,NaN,1.0,N,Other
743293,100559481098,2000-02-01,NaN,8.625,NaN,1.0,359.0,359.0,2030-01-01,19100.0,...,1,S,TX,750,35.0,FRM,NaN,1.0,N,Other


,loan_id,monthly_reporting_period,servicer,interest_rate,current_actual_upb,loan_age,remaining_months_to_legal_maturity,adj_remaining_months_to_maturity,maturity_date,msa,...,num_units,occupancy_status,property_state,zip,mortgage_insurance_percent,product_type,coborrow_credit_score,mortgage_insurance_type,relocation_mortgage_indicator,seller_name
1130128,100023274028,2000-07-01,NaN,8.000,NaN,5.0,355.0,355.0,2030-02-01,12060.0,...,1,P,GA,300,NaN,FRM,782.0,NaN,N,Other
1130129,100185534396,2001-02-01,NaN,7.875,150966.36,14.0,346.0,346.0,2029-12-01,13820.0,...,1,P,AL,352,25.0,FRM,NaN,1.0,N,NaN
1130130,100185534396,2001-03-01,NaN,7.875,150851.35,15.0,345.0,345.0,2029-12-01,13820.0,...,1,P,AL,352,25.0,FRM,NaN,1.0,N,NaN
1130131,100185534396,2001-05-01,NaN,7.875,150619.05,17.0,343.0,343.0,2029-12-01,13820.0,...,1,P,AL,352,25.0,FRM,NaN,1.0,N,NaN
1130132,100204073687,2003-04-01,NaN,7.625,56307.38,39.0,141.0,141.0,2015-01-01,38060.0,...,1,P,AZ,853,NaN,FRM,767.0,NaN,N,Other


,loan_id,monthly_reporting_period,servicer,interest_rate,current_actual_upb,loan_age,remaining_months_to_legal_maturity,adj_remaining_months_to_maturity,maturity_date,msa,...,num_units,occupancy_status,property_state,zip,mortgage_insurance_percent,product_type,coborrow_credit_score,mortgage_insurance_type,relocation_mortgage_indicator,seller_name
1507825,100111651961,2002-09-01,NaN,8.500,39957.83,30.0,330.0,330.0,2030-03-01,26420.0,...,1,S,TX,770,NaN,FRM,678.0,NaN,N,NaN
1507826,100135299882,2000-05-01,NaN,8.125,NaN,3.0,357.0,357.0,2030-02-01,35380.0,...,1,P,LA,704,30.0,FRM,698.0,2.0,N,NaN
1507827,100135299882,2000-08-01,NaN,8.125,128338.64,6.0,354.0,354.0,2030-02-01,35380.0,...,1,P,LA,704,30.0,FRM,698.0,2.0,N,NaN
1507828,100135299882,2001-10-01,NaN,8.125,127054.28,20.0,340.0,340.0,2030-02-01,35380.0,...,1,P,LA,704,30.0,FRM,698.0,2.0,N,NaN
1507829,100135299882,2002-10-01,NaN,8.125,125956.47,32.0,328.0,329.0,2030-02-01,35380.0,...,1,P,LA,704,30.0,FRM,698.0,2.0,N,NaN


...

,loan_id,monthly_reporting_period,servicer,interest_rate,current_actual_upb,loan_age,remaining_months_to_legal_maturity,adj_remaining_months_to_maturity,maturity_date,msa,...,num_units,occupancy_status,property_state,zip,mortgage_insurance_percent,product_type,coborrow_credit_score,mortgage_insurance_type,relocation_mortgage_indicator,seller_name
7195865,100199760845,2002-03-01,NaN,7.625,165481.32,25.0,155.0,155.0,2015-02-01,16980.0,...,1,P,IL,606,NaN,FRM,751.0,NaN,N,Other
7195866,100199760845,2003-03-01,NaN,7.625,157652.25,37.0,143.0,143.0,2015-02-01,16980.0,...,1,P,IL,606,NaN,FRM,751.0,NaN,N,Other
7195867,100199760845,2003-05-01,NaN,7.625,156972.57,39.0,141.0,0.0,2015-02-01,16980.0,...,1,P,IL,606,NaN,FRM,751.0,NaN,N,Other
7195868,100393239071,2002-02-01,NaN,7.875,105902.48,25.0,275.0,242.0,2025-01-01,17460.0,...,1,P,OH,440,NaN,FRM,797.0,NaN,N,Suntrust
7195869,100393239071,2000-04-01,NaN,7.875,NaN,3.0,297.0,295.0,2025-01-01,17460.0,...,1,P,OH,440,NaN,FRM,797.0,NaN,N,Suntrust


,loan_id,monthly_reporting_period,servicer,interest_rate,current_actual_upb,loan_age,remaining_months_to_legal_maturity,adj_remaining_months_to_maturity,maturity_date,msa,...,num_units,occupancy_status,property_state,zip,mortgage_insurance_percent,product_type,coborrow_credit_score,mortgage_insurance_type,relocation_mortgage_indicator,seller_name
7578238,100292236815,2004-08-01,NaN,8.50,54557.51,53.0,307.0,0.0,2030-03-01,12580.0,...,1,P,MD,212,25.0,FRM,NaN,1.0,N,Other
7578239,100292236815,2000-10-01,NaN,8.50,56734.61,7.0,353.0,353.0,2030-03-01,12580.0,...,1,P,MD,212,25.0,FRM,NaN,1.0,N,Other
7578240,100339426081,2000-09-01,NaN,7.75,151604.34,8.0,352.0,352.0,2030-01-01,16980.0,...,1,P,IL,600,NaN,FRM,NaN,NaN,N,NaN
7578241,100442743596,2001-04-01,NaN,7.75,165406.90,13.0,347.0,347.0,2030-03-01,19740.0,...,1,P,CO,801,NaN,FRM,752.0,NaN,Y,NaN
7578242,100442743596,2000-12-01,NaN,7.75,165911.36,9.0,351.0,351.0,2030-03-01,19740.0,...,1,P,CO,801,NaN,FRM,752.0,NaN,Y,NaN


,loan_id,monthly_reporting_period,servicer,interest_rate,current_actual_upb,loan_age,remaining_months_to_legal_maturity,adj_remaining_months_to_maturity,maturity_date,msa,...,num_units,occupancy_status,property_state,zip,mortgage_insurance_percent,product_type,coborrow_credit_score,mortgage_insurance_type,relocation_mortgage_indicator,seller_name
7956403,100125587957,2007-03-01,NaN,8.5,83092.47,84.0,276.0,270.0,2030-03-01,0.0,...,1,P,TX,759,25.0,FRM,766.0,1.0,N,Other
7956404,100125587957,2007-05-01,NaN,8.5,36469.85,86.0,274.0,66.0,2030-03-01,0.0,...,1,P,TX,759,25.0,FRM,766.0,1.0,N,Other
7956405,100125587957,2010-07-01,NaN,8.5,11519.59,124.0,236.0,18.0,2030-03-01,0.0,...,1,P,TX,759,25.0,FRM,766.0,1.0,N,Other
7956406,100125587957,2005-08-01,NaN,8.5,85461.29,65.0,295.0,295.0,2030-03-01,0.0,...,1,P,TX,759,25.0,FRM,766.0,1.0,N,Other
7956407,100125587957,2007-08-01,NaN,8.5,35025.33,89.0,271.0,63.0,2030-03-01,0.0,...,1,P,TX,759,25.0,FRM,766.0,1.0,N,Other


,loan_id,monthly_reporting_period,servicer,interest_rate,current_actual_upb,loan_age,remaining_months_to_legal_maturity,adj_remaining_months_to_maturity,maturity_date,msa,...,num_units,occupancy_status,property_state,zip,mortgage_insurance_percent,product_type,coborrow_credit_score,mortgage_insurance_type,relocation_mortgage_indicator,seller_name
8337488,100019048933,2002-09-01,NaN,7.500,43390.88,32.0,148.0,132.0,2015-01-01,36740.0,...,1,P,FL,347,30.0,FRM,NaN,1.0,N,Other
8337489,100020205696,2000-01-01,NaN,7.625,NaN,5.0,355.0,352.0,2029-08-01,42660.0,...,1,P,WA,980,NaN,FRM,770.0,NaN,N,Other
8337490,100020205696,2001-01-01,NaN,7.625,220517.61,17.0,343.0,335.0,2029-08-01,42660.0,...,1,P,WA,980,NaN,FRM,770.0,NaN,N,Other
8337491,100020205696,2000-12-01,NaN,7.625,220517.61,16.0,344.0,335.0,2029-08-01,42660.0,...,1,P,WA,980,NaN,FRM,770.0,NaN,N,Other
8337492,100179856394,2004-01-01,NaN,8.000,35424.80,47.0,73.0,68.0,2010-02-01,41980.0,...,1,P,PR,9,NaN,FRM,793.0,NaN,N,Other


,loan_id,monthly_reporting_period,servicer,interest_rate,current_actual_upb,loan_age,remaining_months_to_legal_maturity,adj_remaining_months_to_maturity,maturity_date,msa,...,num_units,occupancy_status,property_state,zip,mortgage_insurance_percent,product_type,coborrow_credit_score,mortgage_insurance_type,relocation_mortgage_indicator,seller_name
8719149,100249783497,2000-11-01,NaN,7.625,106247.23,10.0,350.0,345.0,2030-01-01,24340.0,...,1,P,MI,493,NaN,FRM,NaN,NaN,N,NaN
8719150,100282003625,2000-11-01,NaN,8.375,41758.29,8.0,352.0,351.0,2030-03-01,15380.0,...,1,P,NY,140,NaN,FRM,NaN,NaN,N,Other
8719151,100394926109,2003-12-01,NaN,8.000,83824.30,46.0,314.0,287.0,2030-02-01,0.0,...,1,P,OR,978,NaN,FRM,NaN,NaN,N,Other
8719152,100808694286,2001-01-01,NaN,8.125,151753.27,12.0,348.0,348.0,2030-01-01,12060.0,...,1,P,GA,300,12.0,FRM,NaN,1.0,N,Other
8719153,100808694286,2001-12-01,OTHER,8.125,150634.37,23.0,337.0,0.0,2030-01-01,12060.0,...,1,P,GA,300,12.0,FRM,NaN,1.0,N,Other
